In [ ]:
import torch
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import to_networkx
from sklearn.decomposition import PCA

In [ ]:
# Load Cora dataset
dataset = Planetoid(root="data/Cora", name="Cora")
data = dataset[0]

# Convert to NetworkX graph for visualization
G = to_networkx(data, to_undirected=True)

# Color nodes by their label
colors = data.y.numpy()

# Plot the graph
plt.figure(figsize=(10, 7))
nx.draw(G, node_color=colors, cmap=plt.get_cmap("jet"), node_size=50, edge_color="gray", alpha=0.6)
plt.title("Cora Citation Network")
plt.show()

In [ ]:

# 🔹 Define the GAT model
class GATWithAttention(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super(GATWithAttention, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, dropout=dropout, add_self_loops=False)

    def forward(self, x, edge_index, return_attention=False):
        x, alpha1 = self.conv1(x, edge_index, return_attention_weights=return_attention)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x, alpha2 = self.conv2(x, edge_index, return_attention_weights=return_attention)

        if return_attention:
            return x, alpha1, alpha2
        return x
    

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GATWithAttention(dataset.num_node_features, 8, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


# 🔹 Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# 🔹 Evaluation function
@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs

# Evaluate embeddings using PCA
@torch.no_grad()
def visualize_embeddings(model, data, title):
    model.eval()
    out = model(data.x, data.edge_index).cpu().numpy()
    
    # Reduce to 2D with PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(out)

    plt.figure(figsize=(8, 6))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=data.y.cpu().numpy(), cmap="jet", s=20)
    plt.title(title)
    plt.colorbar(label="Class Label")
    plt.show()

# Visualize embeddings before training
visualize_embeddings(model, data, "Initial Node Embeddings (Random)")


# 🔹 Run training
for epoch in range(200):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

# 🔹 Final test accuracy
print(f"Final Test Accuracy: {test()[2]:.4f}")

# Visualize embeddings after training
visualize_embeddings(model, data, "Node Embeddings After GAT Training")